# Imports

In [1]:
import segmentation_models as sm
from segmentation_models.utils import set_trainable
from keras.preprocessing.image import ImageDataGenerator

## error
#from keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers import Adam, SGD

# AttributeError: module 'keras.utils' has no attribute 'get_file' using segmentation_models
sm.set_framework("tf.keras")
sm.framework()

## Using function in this notebook
#from model import *
#from data import *

import numpy as np
import os
import glob
import skimage.io as io
import skimage.transform as trans
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, CSVLogger


Segmentation Models: using `keras` framework.


# To use GPU

In [2]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)


Num GPUs Available 1


# To disable GPU and use CPU

In [ ]:
CUDA_VISIBLE_DEVICES=""

#import os
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

# No GPU found

# Functions

In [3]:
def adjustData(img, mask, flag_multi_class, num_class):
    if(flag_multi_class):
        img = img / 255
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255
        mask = mask /255
        mask[mask >= 0.5] = 1
        mask[mask < 0.5] = 0
    return (img,mask)


In [4]:
data_gen_args = dict(rotation_range=180,
                    width_shift_range=0.07,
                    height_shift_range=0.07,
                    shear_range=0.07,
                    zoom_range=0.07,
                    horizontal_flip=True,
                    fill_mode='nearest')

## To check
#batch_size = 2
#train_samples = 330
#test_samples = 37
#steps_p_epoch = int(train_samples / batch_size)
#val_steps = int(test_samples / batch_size)

nr_epochs = 50 #50


In [5]:
#from skimage.io import imread, imsave

def makeGenerator(data_frame, batch_size, folder_path, img_folder, mask_folder,
                  aug_dict, image_color_mode = "grayscale",
                  mask_color_mode = "grayscale", image_save_prefix = "image", mask_save_prefix = "mask",
                  flag_multi_class = False, num_class = 2, save_to_dir = None, target_size = (256, 256),
                  seed = 42, isTrainGenerator = True):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    If isTrainGenerator True will use aug_dict and make data aumentation
    '''

    #print("\nfolder_path", folder_path)
    #!pwd
    #!ls

    if (isTrainGenerator):
        image_datagen = ImageDataGenerator(**aug_dict)
        mask_datagen = ImageDataGenerator(**aug_dict)
    else:
        image_datagen = ImageDataGenerator()
        mask_datagen = ImageDataGenerator()

    image_generator = image_datagen.flow_from_dataframe(
        data_frame,
        folder_path + img_folder,
        x_col = 'filename',
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix = image_save_prefix,
        seed = seed)

    mask_generator = mask_datagen.flow_from_dataframe(
        data_frame,
        folder_path + mask_folder,
        x_col = 'filename',
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)

    make_generator = zip(image_generator, mask_generator)
    for (img, mask) in make_generator:
        #print("img, mask")
        #!dir

        #print("img", img)
        #print(type(img))
        #print(img.dtype)
        #print(img.shape)
        #print(mask.shape)

        #img2 = img[0, :, :, :]
        #print(img2.shape)

        #mask2 = mask[0, :, :, :]
        #print(mask2.shape)

        #imsave("img.png", img2, check_contrast=False)
        #imsave("mask.png", mask2, check_contrast=False)

        img, mask = adjustData(img, mask, flag_multi_class, num_class)
        yield(img, mask)


In [6]:
#def load_model(DECODER, BACKBONE, input_shape, encoder_weights, encoder_freeze, classes, activation):
def load_model(DECODER, model_args):
    #model = sm.Unet(
    #model = sm.Linknet(
    #model = sm.PSPNet(

    #    backbone_name=BACKBONE, input_shape=(x, x, 3), encoder_weights='imagenet',
    #    encoder_freeze=True, classes=1, activation='sigmoid'
    #)

    ## input_shape – shape of input data/image (H, W, C)
        # Unet and Linknet - H and W of input images should be divisible by factor 32.
        # PSPNet - H and W should be divisible by 6 * downsample_factor and NOT None! - downsample_factor = 8

    if DECODER == "Unet":
        print("\n1 DECODER:", DECODER)
        model = sm.Unet(**model_args)

    elif DECODER == "Linknet":
        print("\n2 DECODER:", DECODER)
        model = sm.Linknet(**model_args)

    elif DECODER == "PSPNet":
        print("\n3 DECODER:", DECODER)
        model = sm.PSPNet(**model_args)
    else :
        raise ValueError("Error DECODER:", DECODER)

    return model


# To check

## Seed in def makeGenerator() diff to train and test
    ###Status = OK

## Seed in def makeGenerator()
    batch_size = 2
    train_samples = len(train_index) # 330
    test_samples = len(test_index) # 37 
    steps_p_epoch = int(train_samples / batch_size)
    val_steps = int(test_samples / batch_size)
    ### Status = OK

## Add fold in file name
    ### Status = ok

## Run code with GPU
    ### Status = OK


# Run

In [ ]:
## Define model
#DECODER = "Unet"
#DECODER = "Linknet"
#DECODER = "PSPNet"

#path = "/media/sda2/home/j/Downloads/tests/recortes_tmp/"
path = "E:/Backes/Segmentacao Linha Plantio CNN/Recortes/"
#path = "/run/media/j/tmp_ntfs/0_t_mest_bkp/tests/Segmentacao Linha Plantio CNN/Recortes/"

#dataset_img_folder = "Base_A"
#dataset_img_folder = "Base_B"
#dataset_img_folder = "Base_C"
#dataset_img_folder = "Base_D"

#dataset_img_folder = "Base_E200"
#dataset_img_folder = "Base_E300"
#dataset_img_folder = "Base_E400"
dataset_img_folder = "Base_E500"

dataset_mask_folder = dataset_img_folder + "_mask"

folds = 10

resultFolder = "results/" + dataset_img_folder + '/'

#save_trans_path = "transformation" #save_to_dir = None
save_trans_path = None

BACKBONE = "vgg16"

import pandas as pd
from sklearn.model_selection import KFold

for i in range(3): #3
    if i == 0:
        DECODER = "Unet"
    elif i == 1:
        DECODER = "Linknet"
    elif i == 2:
        DECODER = "PSPNet"

    #files = sorted(glob.glob(os.path.join(path + dataset_img_folder, "*.png")))
    files = sorted([os.path.basename(x) for x in glob.glob(os.path.join(path + dataset_img_folder, "*.png"))])
    #for name in files:
    #    print(name)

    #The first n_samples % n_splits folds have size n_samples // n_splits + 1,
    #other folds have size n_samples // n_splits, where n_samples is the number of samples
    kf = KFold(n_splits = folds, random_state = 42, shuffle = True)

    nr_fold = 1 #1
    fold_to_jump = 0 #0

    for train_index, test_index in kf.split(files):
        #print("\nTRAIN:", train_index, "\n\nTEST:", test_index)
        #X_train, X_test = X[train_index], X[test_index]
        #y_train, y_test = y[train_index], y[test_index]

        if (nr_fold <= fold_to_jump):
            print("\n\nFold: - nothing to do", nr_fold, end = '')
            nr_fold = nr_fold + 1
            continue

        train_list_files = []
        for it in train_index:
            train_list_files.append(files[it])

        test_list_files = []
        for it in test_index:
            test_list_files.append(files[it])

        ## To check
        batch_size = 2
        train_samples = len(train_index)
        test_samples = len(test_index)
        steps_p_epoch = int(train_samples / batch_size)
        val_steps = int(test_samples / batch_size)

        print("\nFold:", nr_fold, end = '')
        print(" len(train_index):", len(train_index), "len(test_index):", len(test_index))

        print("\ndataset_img_folder:", dataset_img_folder) 
        print("batch_size:", batch_size)
        print("train_samples:", train_samples)
        print("test_samples:", test_samples)
        print("steps_p_epoch:", steps_p_epoch)
        print("val_steps:", val_steps)

        train_data_frame = pd.DataFrame(train_list_files, columns = ['filename'])
        test_data_frame = pd.DataFrame(test_list_files, columns = ['filename'])
        #print("\ntrain_data_frame\n", train_data_frame, "\n\ntest_data_frame\n", test_data_frame)

        preprocess_input = sm.get_preprocessing(BACKBONE)

        # size - target_size?: 240 PSPNet, 256 Unet and Linknet, 288 All
        if DECODER == "Unet" or DECODER == "Linknet":
            size = 256 # 240 # 256 # 288 ## target_size
        elif DECODER == "PSPNet":
            size = 240
        else:
            raise ValueError("Error DECODER:", DECODER)

        print("\ntarget_size:", size, size)

        trainGene = makeGenerator(train_data_frame, batch_size, path, dataset_img_folder, dataset_mask_folder,
                                  aug_dict = data_gen_args, save_to_dir = save_trans_path, image_color_mode = "rgb",
                                  isTrainGenerator = True, target_size = (size, size))

        testGene = makeGenerator(test_data_frame, batch_size, path, dataset_img_folder, dataset_mask_folder,
                                 aug_dict = '', save_to_dir = save_trans_path, image_color_mode = "rgb",
                                 isTrainGenerator = False, target_size = (size, size))

        ## input_shape – shape of input data/image (H, W, C)
            # Unet and Linknet - H and W of input images should be divisible by factor 32.
            # PSPNet - H and W should be divisible by 6 * downsample_factor and NOT None! - downsample_factor = 8

        ## add input_shape = (size, size, 3) beacuse PSPNet
        data_load_model_args = dict(backbone_name = BACKBONE, input_shape = (size, size, 3),
                                    encoder_weights='imagenet', encoder_freeze = True,
                                    classes = 1, activation = 'sigmoid')

        model = load_model(DECODER, data_load_model_args)

        if nr_fold == 1:
            model.summary()

        #model = sm.Unet(
        #model = sm.Linknet(
        #model = sm.PSPNet(
        #model = sm.PSPNet(backbone_name=BACKBONE, input_shape=(240, 240, 3),
        #model = sm.PSPNet(backbone_name=BACKBONE, input_shape=(288, 288, 3),

        #model = sm.Unet(
        #model = sm.Linknet(
        #model = sm.PSPNet(
            #backbone_name=BACKBONE, input_shape=(x, x, 3),
            #encoder_weights='imagenet',
            #encoder_freeze=True,
            #classes=1,
            #activation='sigmoid'
        #)

        opt = Adam(learning_rate = 1e-3)

        model.compile(opt,
            loss=sm.losses.jaccard_loss,
            metrics=[sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), sm.metrics.precision]
        )

        csv_logger = CSVLogger(resultFolder + BACKBONE + "_" + DECODER  + "_" + dataset_img_folder + ".csv", append=True)

        model_checkpoint = ModelCheckpoint(resultFolder + BACKBONE + "_" + DECODER + "_" + dataset_img_folder + "_" +
                                           str(nr_fold) + ".hdf5", monitor = "loss", verbose = 1, save_best_only=True)

        history = model.fit_generator(generator = trainGene,
            validation_data = testGene,
            validation_steps = val_steps,
            steps_per_epoch = steps_p_epoch,
            epochs = nr_epochs,
            callbacks = [model_checkpoint, csv_logger])

        nr_fold = nr_fold + 1

print("\n\nAll Done!\n")
